# Download DEM as Tiff

In [ ]:
! wget https://sbnarchive.psi.edu/pds3/dawn/fc/DWNCLSPG_2/EXTRAS/DTM/CE_LAMO_L_IKAPATI_EQU_DTM.TIF

# Read in metadata and plot heights

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt


NODATA_VALUE = -32768
DTM_PATH = "CE_LAMO_L_AHUNAMONS_EQU_DTM.TIF"

src = rasterio.open(DTM_PATH)
z_values = np.flipud(src.read(1) * 1.0)
print(src.meta)

print(f"Number of points with no data: {np.sum(z_values == NODATA_VALUE)}")
z_values[z_values == NODATA_VALUE] = np.nan
#z_values = z_values[1000:-1000, 500:-500]

plt.imshow(z_values, cmap="gray")
#plt.axis("off")
plt.show()

## Values for Ceres

In [ ]:
# Define constants for convertion.
# Ref: https://sbnarchive.psi.edu/pds3/dawn/fc/DWNCLSPG_2/CATALOG/FC2_CERES_LAMO_DTM_SPG_DS.CAT
RADIUS = 470000  # meters
SCALING_FACTOR = 1.0
LATERAL_SPACING = 32.0431542  # meters
#LAT_CENTER, LON_CENTER = -10.0 * np.pi / 180, 315.0 * np.pi / 180  # AHUNA MONS
LAT_CENTER, LON_CENTER =  33.0 * np.pi / 180,  43.0 * np.pi / 180  # IKAPATI

# Convert to point cloud

In [ ]:
import visu3d as v3d


# Get the dimensions of the terrain data.
height, width = z_values.shape

# Generate x and y grid coordinates and scale by pixel resolution.
x_coords, y_coords = np.meshgrid(np.arange(width), np.arange(height))
x_coords = x_coords.astype(float) * LATERAL_SPACING
y_coords = y_coords.astype(float) * LATERAL_SPACING
print(x_coords.max(), y_coords.max())

# Compute normalization constants. 
c_x = np.pi * RADIUS
c_y = np.pi * RADIUS / 2

# Center and normalize coordinates.
x_normalized = (x_coords - x_coords.max() / 2) / c_x
y_normalized = (y_coords - y_coords.max() / 2) / c_y

# Map normalized coordinates to spherical coordinates
lon = x_normalized * np.pi + LON_CENTER  # longitude in radians from -π to π
lat = y_normalized * np.pi / 2 + LAT_CENTER  # latitude in radians from -π/2 to π/2
print("Longitude range:", np.rad2deg(np.nanmin(lon)), np.rad2deg(np.nanmax(lon)))
print("Latitude range:", np.rad2deg(np.nanmin(lat)), np.rad2deg(np.nanmax(lat)))

# Convert spherical coordinates to Cartesian coordinates
radii = z_values * SCALING_FACTOR + RADIUS
x_sphere = radii * np.cos(lat) * np.cos(lon) / 1000
y_sphere = radii * np.cos(lat) * np.sin(lon) / 1000
z_sphere = radii * np.sin(lat) / 1000

print(np.nanmax(x_sphere) - np.nanmin(x_sphere))
print(np.nanmax(y_sphere) - np.nanmin(y_sphere))
print(np.nanmax(z_sphere) - np.nanmin(z_sphere))

# Combine x, y, and z coordinates into a single array of 3D coordinates
vertices = np.vstack((x_sphere.flatten(), y_sphere.flatten(), z_sphere.flatten())).T

# Plot scene.
pc = v3d.Point3d(p=vertices)
v3d.make_fig([pc])

In [ ]:
# Generate triangular faces

faces = []
for i in range(height - 1):
    print(i)
    for j in range(width - 1):
        # Define indices of the corners of the cell
        idx_tl = i * width + j          # Top-left
        idx_tr = i * width + j + 1      # Top-right
        idx_bl = (i + 1) * width + j    # Bottom-left
        idx_br = (i + 1) * width + j + 1 # Bottom-right
        
        # Add two triangles for the cell
        faces.append([idx_tl, idx_tr, idx_bl])  # Triangle 1
        faces.append([idx_tr, idx_br, idx_bl])  # Triangle 2
faces = np.asarray(faces)

In [ ]:
import open3d as o3d

# Create an Open3D TriangleMesh
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(faces)

# Compute normals for better visualization
mesh.compute_vertex_normals()


o3d.io.write_triangle_mesh(DTM_PATH.replace("_EQU_DTM", "_MESH").replace(".TIF", ".ply"), mesh, write_ascii=False)

# Save pointcloud as PLY

In [ ]:
import open3d as o3d

valid_ind = ~np.isnan(z_sphere.flatten())
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(vertices[valid_ind])

o3d.io.write_point_cloud(DTM_PATH.replace("_EQU_DTM", "_PC").replace(".TIF", ".ply"), pc, write_ascii=False)